In [1]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("bghira/free-to-use-pixelart")

/Users/harry.sadler/Library/Caches/pypoetry/virtualenvs/pixelart-dataset-exploration-iWfMpuw5-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 7273/7273 [00:00<00:00, 191449.45 examples/s]


In [3]:
train = ds['train']
df = pd.DataFrame(train)
df.head(3)

,subset,sequence,subset-sequence-element,title,description,views,filename,pixel_size,has_watermark,image_hash,image_url,full_image_url,likes_count,comments_count,width,height,date_created,content_warning,warning,liked
0,highlighted,1,highlighted.1.1,Highlander,,36,sr2e3cecf2e99aws3-1 Highlander.png,6,False,sr2e3cecf2e99aws3,https://art.pixilart.com/thumb/sr2e3cecf2e99aw...,https://art.pixilart.com/sr2e3cecf2e99aws3.png,9,0,200,250,05/25/2024,None,False,False
1,highlighted,1,highlighted.1.2,Dungeon Key,#challenge,166,sr26fa2adb2a6aws3-2 Dungeon Key.png,14,False,sr26fa2adb2a6aws3,https://art.pixilart.com/thumb/sr26fa2adb2a6aw...,https://art.pixilart.com/sr26fa2adb2a6aws3.gif,49,2,54,100,05/25/2024,None,False,False
2,highlighted,1,highlighted.1.3,Mecha1old,Old mecha i did back in 2022,117,sr20b7ff421b0aws3-3 Mecha1old.png,11,False,sr20b7ff421b0aws3,https://art.pixilart.com/thumb/sr20b7ff421b0aw...,https://art.pixilart.com/sr20b7ff421b0aws3.gif,39,0,128,128,05/25/2024,None,False,False
